In [1]:
import json
import copy
import pandas as pd
import numpy as np

from pathlib import Path


### Assegnare a ogni (stazione, anno, mese) un punteggio di anomalia climatica interpretabile, basato su deviazioni multivariate rispetto al comportamento storico atteso.


In [2]:
data_path = Path("../data/processed/monthly.parquet")


In [3]:
CORE_FEATURES = [
    "precipitation_sum",
    "precipitation_rainy_days",
    "temperature_min_mean",
    "temperature_mean_mean",
    "temperature_max_mean",
    "humidity_mean_mean",
    "wind_speed_mean_mean",
    "solar_radiation_mean",
    "pressure_mean_mean",
]

FEATURE_TO_COVERAGE = {
    "precipitation_sum": "precipitation_coverage",
    "precipitation_rainy_days": "precipitation_coverage",
    "temperature_min_mean": "temperature_min_coverage",
    "temperature_mean_mean": "temperature_mean_coverage",
    "temperature_max_mean": "temperature_max_coverage",
    "humidity_mean_mean": "humidity_mean_coverage",
    "wind_speed_mean_mean": "wind_speed_mean_coverage",
    "solar_radiation_mean": "solar_radiation_coverage",
    "pressure_mean_mean": "pressure_mean_coverage",
}

In [4]:
monthly_df = pd.read_parquet(data_path)
display(monthly_df.head(2))
print(monthly_df.shape[0])

,station_name,year,month,precipitation_sum,precipitation_max,precipitation_mean,temperature_min_mean,temperature_min_min,temperature_min_max,temperature_min_std,...,temperature_mean_coverage,temperature_max_coverage,humidity_min_coverage,humidity_mean_coverage,humidity_max_coverage,wind_speed_mean_coverage,wind_speed_max_coverage,wind_direction_max_coverage,solar_radiation_coverage,pressure_mean_coverage
0,Gemona del Friuli,1999,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Gemona del Friuli,1999,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


2172


In [5]:
# params (MVP)
MIN_DAYS_ROWS = 20
MIN_MEAN_COVERAGE = 0.70
MIN_FEATURES_PRESENT = 6  # su 9 core sopra (puoi alzare/abbassare)

# keep only features present in df (safety)
core = [c for c in CORE_FEATURES if c in monthly_df.columns]
cov_cols = [FEATURE_TO_COVERAGE[c] for c in core if FEATURE_TO_COVERAGE.get(c) in monthly_df.columns]

# 1) days coverage
days_ok = monthly_df["n_days_rows"] >= MIN_DAYS_ROWS

# 2) feature availability (NaN-aware)
monthly_df["n_features_present"] = monthly_df[core].notna().sum(axis=1)
feat_ok = monthly_df["n_features_present"] >= MIN_FEATURES_PRESENT

# 3) data coverage (mean over mapped coverage cols)
monthly_df["mean_coverage_core"] = monthly_df[cov_cols].mean(axis=1, skipna=True)
cov_ok = monthly_df["mean_coverage_core"] >= MIN_MEAN_COVERAGE

# final gating
monthly_df["is_evaluable"] = days_ok & feat_ok & cov_ok

# reason (for debugging/transparency)
monthly_df["gate_reason"] = "OK"
monthly_df.loc[~days_ok, "gate_reason"] = "LOW_DAYS"
monthly_df.loc[days_ok & ~feat_ok, "gate_reason"] = "FEW_FEATURES"
monthly_df.loc[days_ok & feat_ok & ~cov_ok, "gate_reason"] = "LOW_COVERAGE"

# quick report
print("Evaluable:", int(monthly_df["is_evaluable"].sum()), "/", len(monthly_df))
print(monthly_df["gate_reason"].value_counts())

Evaluable: 1493 / 2172
gate_reason
OK              1493
LOW_COVERAGE     470
FEW_FEATURES     200
LOW_DAYS           9
Name: count, dtype: int64


### We now compute the z-score

In [6]:
working_df = copy.deepcopy(monthly_df)
working_df = working_df[working_df["is_evaluable"]]
display(working_df.head(1))

,station_name,year,month,precipitation_sum,precipitation_max,precipitation_mean,temperature_min_mean,temperature_min_min,temperature_min_max,temperature_min_std,...,humidity_max_coverage,wind_speed_mean_coverage,wind_speed_max_coverage,wind_direction_max_coverage,solar_radiation_coverage,pressure_mean_coverage,n_features_present,mean_coverage_core,is_evaluable,gate_reason
7,Gemona del Friuli,1999,8,257.2,105.8,10.288,17.144,13.6,19.9,1.622005,...,0.806452,0.806452,0.806452,0.806452,0.806452,0.548387,9,0.777778,True,OK


In [7]:
SCALE = 1.4826  # standard scaling for MAD

def compute_baseline(group, features):
    med = group[features].median()
    mad = (group[features] - med).abs().median() * SCALE

    mad = mad.replace(0, np.nan)

    return pd.concat(
        [
            med.add_suffix("_median"),
            mad.add_suffix("_mad"),
        ]
    )

In [8]:
baseline = (
    working_df
    .groupby("station_name", group_keys=False)
    .apply(compute_baseline, features=CORE_FEATURES)
)

/var/folders/7w/jn4ypvxs6g303tpt3jxt73xr0000gp/T/ipykernel_74693/1777349039.py:4: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compute_baseline, features=CORE_FEATURES)


In [9]:
df_z = working_df.merge(
    baseline.reset_index(),
    on="station_name",
    how="left"
)

In [10]:
Z_COLS = []

for f in CORE_FEATURES:
    med_col = f"{f}_median"
    mad_col = f"{f}_mad"
    z_col = f"{f}_z"

    df_z[z_col] = (df_z[f] - df_z[med_col]) / df_z[mad_col]
    Z_COLS.append(z_col)

df_z[Z_COLS].describe()

,precipitation_sum_z,precipitation_rainy_days_z,temperature_min_mean_z,temperature_mean_mean_z,temperature_max_mean_z,humidity_mean_mean_z,wind_speed_mean_mean_z,solar_radiation_mean_z,pressure_mean_mean_z
count,915.000000,915.000000,1493.000000,1493.000000,1493.000000,1471.000000,1486.000000,1485.000000,1421.000000
mean,0.277655,0.028921,0.012305,0.014350,0.005522,-0.164384,0.072326,-0.045053,-0.048648
std,1.235722,1.047332,0.739957,0.746165,0.749454,1.145441,1.111147,0.750832,1.505250
min,-1.351348,-2.697963,-1.645361,-1.688573,-1.725362,-5.883058,-3.720608,-1.509112,-33.803410
25%,-0.597141,-0.674491,-0.661606,-0.654866,-0.660298,-0.780944,-0.659875,-0.729717,-0.649491
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.845292,0.674491,0.694727,0.706687,0.685319,0.611546,0.677962,0.607754,0.696789
max,5.978473,3.372454,1.552164,1.557866,1.607056,2.701990,6.498143,1.776749,4.628914


In [11]:
ZCAP = 10  # soglia robusta (8–10 vanno bene)

for zc in Z_COLS:
    df_z[zc] = df_z[zc].clip(-ZCAP, ZCAP)

In [12]:
df_z["n_features_used"] = df_z[Z_COLS].notna().sum(axis=1)

df_z["anomaly_score"] = (
    df_z[Z_COLS]
    .abs()
    .mean(axis=1, skipna=True)
)

df_z[["station_name", "year", "month", "anomaly_score", "n_features_used"]].head()

,station_name,year,month,anomaly_score,n_features_used
0,Gemona del Friuli,1999,8,1.702879,9
1,Gemona del Friuli,1999,9,0.448480,9
2,Gemona del Friuli,1999,10,0.506324,9
3,Gemona del Friuli,1999,11,0.910945,9
4,Gemona del Friuli,1999,12,1.160039,9


In [13]:
df_z["anomaly_score"].describe()

count    1493.000000
mean        0.761333
std         0.294515
min         0.137634
25%         0.565588
50%         0.721557
75%         0.919556
max         2.018398
Name: anomaly_score, dtype: float64

In [14]:
# percentile scelto
PCTL = 0.99

# soglia per stazione
thresholds = (
    df_z
    .groupby("station_name")["anomaly_score"]
    .quantile(PCTL)
    .rename("threshold_p99")
    .reset_index()
)

thresholds.head()

,station_name,threshold_p99
0,Gemona del Friuli,1.555479
1,Lignano,1.536825
2,Monte Lussari,1.487606
3,Monte Matajur,1.651591
4,Monte Zoncolan,1.585139


In [15]:
df_a = df_z.merge(
    thresholds,
    on="station_name",
    how="left"
)

df_a["is_anomaly"] = df_a["anomaly_score"] >= df_a["threshold_p99"]

df_a[["station_name", "year", "month", "anomaly_score", "threshold_p99", "is_anomaly"]].head()

,station_name,year,month,anomaly_score,threshold_p99,is_anomaly
0,Gemona del Friuli,1999,8,1.702879,1.555479,True
1,Gemona del Friuli,1999,9,0.448480,1.555479,False
2,Gemona del Friuli,1999,10,0.506324,1.555479,False
3,Gemona del Friuli,1999,11,0.910945,1.555479,False
4,Gemona del Friuli,1999,12,1.160039,1.555479,False


In [16]:
# % anomalie globale
pct_global = df_a["is_anomaly"].mean() * 100

# % anomalie per stazione
pct_by_station = (
    df_a
    .groupby("station_name")["is_anomaly"]
    .mean()
    .mul(100)
    .sort_values(ascending=False)
)

print(f"Anomalie globali: {pct_global:.2f}%")
pct_by_station.head(10)

Anomalie globali: 1.21%


station_name
Gemona del Friuli    1.290323
Monte Zoncolan       1.230769
Piancavallo          1.190476
Monte Matajur        1.185771
Lignano              1.153846
Monte Lussari        1.075269
Name: is_anomaly, dtype: float64

In [17]:
TOP_K = 5

def top_features_row(row, features):
    contrib = []
    for f in features:
        z = row.get(f"{f}_z")
        if pd.notna(z):
            contrib.append((f, float(abs(z)), float(z), None if pd.isna(row.get(f)) else float(row.get(f))))
    contrib.sort(key=lambda x: x[1], reverse=True)
    # salvo: feature, |z|, z, raw_value
    return json.dumps(contrib[:TOP_K])

df_a["top_features"] = df_a.apply(lambda r: top_features_row(r, CORE_FEATURES), axis=1)

In [19]:
df_a[df_a.is_anomaly].head(5)[
    ["station_name", "year", "month", "anomaly_score", "top_features"]
]

,station_name,year,month,anomaly_score,top_features
0,Gemona del Friuli,1999,8,1.702879,"[[""pressure_mean_mean"", 10.0, -10.0, 869.67647..."
15,Gemona del Friuli,2000,11,1.558443,"[[""precipitation_sum"", 4.732328254958205, 4.73..."
150,Gemona del Friuli,2012,2,1.572123,"[[""humidity_mean_mean"", 3.2495609047909344, -3..."
196,Gemona del Friuli,2015,12,1.708039,"[[""pressure_mean_mean"", 4.587290198259552, 4.5..."
461,Lignano,2012,2,1.612550,"[[""wind_speed_mean_mean"", 4.659374325396652, 4..."


In [21]:
OUT_PATH = "../data/processed/monthly_anomalies.parquet"

cols_to_save = [
    "station_name", "year", "month",
    "anomaly_score", "threshold_p99", "is_anomaly",
    "n_features_used", "mean_coverage_core",
    "top_features"
]

df_a[cols_to_save].to_parquet(OUT_PATH, index=False)

print("Saved:", OUT_PATH)

Saved: ../data/processed/monthly_anomalies.parquet
